In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import data and display 
df = pd.read_csv('time-series/zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [3]:
# more information on the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


We can see that there are more than 14,000 rows and 272 columns.  From printing the top couple of rows above, we can see that our data is presented in wide format.

In [4]:
# look at dataset, prior to dates to get a sense of what else is provided
primary_cols = df.iloc[:, :7]
primary_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RegionID    14723 non-null  int64 
 1   RegionName  14723 non-null  int64 
 2   City        14723 non-null  object
 3   State       14723 non-null  object
 4   Metro       13680 non-null  object
 5   CountyName  14723 non-null  object
 6   SizeRank    14723 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 805.3+ KB


In [5]:
# summarize dataset further
# how many zipcodes represented, states, etc. 
# following this, try to plot on a map based on zipcode and max price / min price based on columns

In [6]:
# number of unique zipcodes (RegionID)
df['RegionID'].nunique()

14723

In [7]:
df['RegionName'].nunique()

14723

We can see there are just over 14,000 unique entries, and each row in our dataset represents one zipcode

In [8]:
# number of cities and states represented
num_cities = df['City'].nunique()
num_states = df['State'].nunique()

print(f'Number of Cities in Dataset: {num_cities}')
print(f'Number of States in Dataset: {num_states}')

Number of Cities in Dataset: 7554
Number of States in Dataset: 51


Looking at the above, we can see that there are 51 unique states represented in our dataset, which can't be as the US only has a total of 50 states.  Look further to see if we can find the outlier.

In [9]:
df['State'].unique()

array(['IL', 'TX', 'NY', 'CA', 'FL', 'TN', 'NC', 'GA', 'DC', 'MO', 'OK',
       'AZ', 'NJ', 'MD', 'VA', 'WA', 'OH', 'MI', 'MA', 'KS', 'NM', 'CT',
       'NV', 'PA', 'CO', 'OR', 'IN', 'SC', 'KY', 'AR', 'ND', 'MN', 'AL',
       'DE', 'LA', 'MS', 'ID', 'MT', 'HI', 'WI', 'UT', 'ME', 'SD', 'WV',
       'IA', 'RI', 'NE', 'WY', 'AK', 'NH', 'VT'], dtype=object)

From quick glance, we see that DC is represented as a state.  For our purposes, we will leave this. 

In [10]:
# number of metros and CountyNames represented
num_metros = df['Metro'].nunique()
num_counties = df['CountyName'].nunique()

print(f'Number of Metros in Dataset: {num_metros}')
print(f'Number of Counties in Dataset: {num_counties}')

Number of Metros in Dataset: 701
Number of Counties in Dataset: 1212


701 metros represented and 1212 counties represented within this dataset

In [11]:
# explore SizeRank
df['SizeRank'].nunique()

14723

In [12]:
# we can see each is ranked according to size, only one size rank per row

In [13]:
# plot zipcodes using folium
import folium

In [14]:
# create a new map object
m = folium.Map(location=(40.0150, -105.2705), zoom_start=3)

# save as an interactive html file
# m.save('circle_map.html')

In [15]:
# plot map
m

In [16]:
# function to get latitude and longitude from given zipcode
df['City'][1]

'McKinney'

In [18]:
# import geocoder to convert zipcode to lat/long
import geocoder

In [21]:
test = geocoder.google('Mountain View, CA')

None


# 2. Data Preprocessing

In [6]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [7]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [8]:
# convert our dataframe
datetimes = get_datetimes(date_cols)
datetimes

DatetimeIndex(['1996-06-01', '1996-07-01', '1996-08-01', '1996-09-01',
               '1996-10-01', '1996-11-01', '1996-12-01', '1997-01-01',
               '1997-02-01', '1997-03-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=263, freq=None)

In [9]:
# set parameters for visualization
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

import matplotlib
matplotlib.rc('font', **font)

In [10]:
# melt data
melted = melt_data(df)
melted.head()

,value
time,
1996-04-01,118299.123063
1996-05-01,118419.044139
1996-06-01,118537.423268
1996-07-01,118653.069278
1996-08-01,118780.254312


In [11]:
melted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 1996-04-01 to 2018-04-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   265 non-null    float64
dtypes: float64(1)
memory usage: 4.1 KB


In [12]:
# concatenate with primary columns
preprocessed = pd.concat([primary_cols, melted], axis=1)
preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14988 entries, 0 to 2018-04-01 00:00:00
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RegionID    14723 non-null  float64
 1   RegionName  14723 non-null  float64
 2   City        14723 non-null  object 
 3   State       14723 non-null  object 
 4   Metro       13680 non-null  object 
 5   CountyName  14723 non-null  object 
 6   SizeRank    14723 non-null  float64
 7   value       265 non-null    float64
dtypes: float64(4), object(4)
memory usage: 1.0+ MB
